In [1]:
import pandas as pd
import spotipy

# Authentication with Spotify

In [2]:
# Client Flow - to get features and analysis from tracks
# Authenticate my user from Spotify

CLIENT_ID = "5424fe3bb2b646c8a2144d03d41f22c7"
CLIENT_SECRET = "f13ad8573f3444908afdf6a6199db827"
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=token)

# Read toptracks_ratings.csv from file 

In [3]:
df_toptracks = pd.read_csv("./dataset/toptracks_ratings.csv")
df_toptracks.head()

,id,name,range,rating
0,7L5IwfKB6W0tadcSh9wlyH,Ouverture,short_term,5.393939
1,0tAZi3X7dUdd7m8OXB8pMA,Shadow,short_term,5.272727
2,1XMDIKQbV30WJPKLMN6MKv,INSTRUCTION,short_term,4.969697
3,1XZdwzd8DTDvkjVc0eJ9BI,Wildlife Analysis,short_term,4.909091
4,1f4cKwcKfNiLbQr8x2tZ3C,Melt!,short_term,4.848485


# Get track features

In [10]:
def get_features(track_id_list, features_list):
    # get track features from spotify. gets a track id_list and features to extract
    # return a dataframe with track features and id as rows.
    
    # dict and dataframe to store features
    dict_features = {}
    df_features = pd.DataFrame()

    for track_id in track_id_list:
        # make API call and get features from spotify web API
        audio_features = sp.audio_features(track_id)[0]
        # add "id_copy" to double check that the right features are added to the right tracks
        dict_features["id_copy"] = track_id
               
        # add all the wanted features into a dict
        for feature in features_list:
            dict_features[feature] = audio_features[feature]

        # create dataframe of feature dict
        df_track = pd.DataFrame(dict_features, index=[0])
        
        #add to df_features dataframe
        df_features = pd.concat([df_features, df_track], ignore_index=True)
        
    return df_features

In [11]:
# features we want to extract
features_list = [
    "energy", 
    "danceability", 
    "loudness", 
    "valence", 
    "tempo", 
    "mode", 
    "speechiness", 
    "acousticness", 
    "instrumentalness", 
    "duration_ms", 
    "time_signature", 
    "tempo"
]

# get all track ids from dataset
id_list = list(df_toptracks["id"])

# get features from tracks in a new dataframe
df_features = get_features(id_list, features_list)
df_features.head()

,id_copy,energy,danceability,loudness,valence,tempo,mode,speechiness,acousticness,instrumentalness,duration_ms,time_signature
0,7L5IwfKB6W0tadcSh9wlyH,0.3360,0.4960,-19.440,0.1850,129.006,1,0.0513,0.95500,0.912,322173,4
1,0tAZi3X7dUdd7m8OXB8pMA,0.2750,0.0832,-15.256,0.0334,170.316,0,0.0347,0.88700,0.853,558267,1
2,1XMDIKQbV30WJPKLMN6MKv,0.3550,0.4910,-12.480,0.0369,145.405,1,0.0354,0.38200,0.887,262733,4
3,1XZdwzd8DTDvkjVc0eJ9BI,0.0204,0.1550,-31.212,0.2780,79.755,0,0.0462,0.99500,0.936,75627,4
4,1f4cKwcKfNiLbQr8x2tZ3C,0.9190,0.7850,-13.059,0.4430,131.037,1,0.0552,0.00772,0.916,214307,4


# Add features to df_toptracks

In [14]:
df_toptracks_features = pd.concat([df_toptracks, df_features], ignore_index=False, axis=1)
df_toptracks_features

,id,name,range,rating,id_copy,energy,danceability,loudness,valence,tempo,mode,speechiness,acousticness,instrumentalness,duration_ms,time_signature
0,7L5IwfKB6W0tadcSh9wlyH,Ouverture,short_term,5.393939,7L5IwfKB6W0tadcSh9wlyH,0.3360,0.4960,-19.440,0.1850,129.006,1,0.0513,0.95500,0.912000,322173,4
1,0tAZi3X7dUdd7m8OXB8pMA,Shadow,short_term,5.272727,0tAZi3X7dUdd7m8OXB8pMA,0.2750,0.0832,-15.256,0.0334,170.316,0,0.0347,0.88700,0.853000,558267,1
2,1XMDIKQbV30WJPKLMN6MKv,INSTRUCTION,short_term,4.969697,1XMDIKQbV30WJPKLMN6MKv,0.3550,0.4910,-12.480,0.0369,145.405,1,0.0354,0.38200,0.887000,262733,4
3,1XZdwzd8DTDvkjVc0eJ9BI,Wildlife Analysis,short_term,4.909091,1XZdwzd8DTDvkjVc0eJ9BI,0.0204,0.1550,-31.212,0.2780,79.755,0,0.0462,0.99500,0.936000,75627,4
4,1f4cKwcKfNiLbQr8x2tZ3C,Melt!,short_term,4.848485,1f4cKwcKfNiLbQr8x2tZ3C,0.9190,0.7850,-13.059,0.4430,131.037,1,0.0552,0.00772,0.916000,214307,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,2OqtZbITDWCFUHAT9fmdin,Choses nouvelles,long_term,4.242424,2OqtZbITDWCFUHAT9fmdin,0.3830,0.6690,-12.993,0.5690,140.011,0,0.0385,0.46500,0.856000,234600,4
234,6EPRKhUOdiFSQwGBRBbvsZ,Ace of Spades,long_term,4.181818,6EPRKhUOdiFSQwGBRBbvsZ,0.9100,0.3420,-7.748,0.5470,140.452,1,0.0758,0.00004,0.000121,166360,4
235,4Jj8pWRyVjh0KIJLrcreRa,Here We Go Jack,long_term,4.121212,4Jj8pWRyVjh0KIJLrcreRa,0.3090,0.6790,-10.606,0.4730,163.979,1,0.0320,0.58500,0.907000,237938,4
236,3v9g8iM3v5irWQHqFWaDSo,Cómo Te Quiero,long_term,4.060606,3v9g8iM3v5irWQHqFWaDSo,0.2770,0.6080,-11.917,0.4370,126.501,1,0.0282,0.62200,0.896000,242536,4


# Save to disk

In [15]:
# add to csv
df_toptracks_features.to_csv("./dataset/toptracks_ratings_features.csv", index=False)